In [2]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import * 
from pyspark.sql.window import Window 

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
data = spark.read.parquet("C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\Dataset\\log_search\\20220601")

In [5]:
data.printSchema()

root
 |-- eventID: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- category: string (nullable = true)
 |-- proxy_isp: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- networkType: string (nullable = true)
 |-- action: string (nullable = true)
 |-- userPlansMap: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [6]:
def process_log_search(data):
    data = data.select('user_id','keyword')
    data = data.groupBy('user_id','keyword').count()
    data = data.withColumnRenamed('count','TotalSearch')
    data = data.orderBy('user_id',ascending = False )
    window = Window.partitionBy('user_id').orderBy(col('TotalSearch').desc())
    data = data.withColumn('Rank',row_number().over(window))
    data = data.filter(col('Rank') == 1)
    data = data.withColumnRenamed('keyword','Most_Search')
    data = data.select('user_id','Most_Search')
    return data 

In [7]:
df = spark.read.parquet("C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\Dataset\\log_search\\20220601")
data = process_log_search(df)

In [8]:
data.show()

+--------+--------------------+
| user_id|         Most_Search|
+--------+--------------------+
|99776428|         truyền hình|
|99779724|   bohemian rhapsody|
|99780748|Liên Minh Công Lý...|
|99780755|  thiên nga bóng đêm|
|99781177|                null|
|99781527|                tieu|
|99785075|              avatar|
| 9978598| THAN THOAI SISIPHUT|
|99786338|             HARY PO|
| 9978695|           chàng hậu|
|99787122|     truc tieptennis|
|99787882|      yuksek sosyete|
|99789765|   yêu từng centimet|
|99790610|   TAN BANG PHONG TH|
|99794377|cô nàng trong trắ...|
|99794952|          KINNPORCHE|
|99797681|             running|
|99800395|hạnh phúc nhỏ của...|
|99801013|cuộc chiến thượng...|
|99801086|khủng hoang hôn nhân|
+--------+--------------------+
only showing top 20 rows



In [23]:
df.select('keyword').groupBy('keyword').count().orderBy('count',ascending = False).show(10,truncate=False)

+--------------------------------------------+-----+
|keyword                                     |count|
+--------------------------------------------+-----+
|null                                        |5048 |
|Liên Minh Công Lý: Phiên bản của Zack Snyder|724  |
|fairy tail                                  |517  |
|cuộc chiến thượng lưu                       |416  |
|yêu nhầm chị dâu                            |335  |
|Thử Thách Thần Tượng                        |294  |
|running man                                 |292  |
|siêu nhân                                   |291  |
|kẻ quyến rũ đại tài                         |284  |
|sao băng                                    |282  |
+--------------------------------------------+-----+
only showing top 10 rows



In [19]:
t7 = spark.read.parquet("C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\Dataset\\log_search\\20220714")


In [22]:
t7.select('keyword').groupBy('keyword').count().orderBy('count',ascending = False).show(10,truncate=False)

+--------------------------------------------+-----+
|keyword                                     |count|
+--------------------------------------------+-----+
|null                                        |4266 |
|Liên Minh Công Lý: Phiên bản của Zack Snyder|714  |
|fairy tail                                  |595  |
|thiên nga bóng đêm                          |451  |
|tinh hà xán lạn                             |423  |
|Thử Thách Thần Tượng                        |396  |
|yêu tinh                                    |376  |
|Kẻ Trộm Mặt Trăng: Minions                  |364  |
|cô vợ bắt buộc                              |363  |
|chàng hậu                                   |334  |
+--------------------------------------------+-----+
only showing top 10 rows

